## Setup

In [1]:
import io
import numpy as np

In [2]:
import os
import cv2

In [3]:
try:
  %tensorflow_version 2.x
except:
  pass

import tensorflow as tf

In [4]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  2


In [5]:
tf.__version__

'2.1.0'

In [6]:
tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [7]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]

In [8]:
#pip install typeguard

In [9]:
#!pip install -q tensorflow_datasets
#!pip install -q --no-deps tensorflow-addons~=0.7

In [10]:
import tensorflow_addons as tfa
import tensorflow_datasets as tfds

In [18]:
# Read Dataset. Split into training and test set
def get_train_test_dataset(train_test_ratio):
    data_src = './data_repository/Veri776/'
    X = []
    y = []
    for directory in os.listdir(data_src):
        #print(directory)
        try:
            for pic in os.listdir(os.path.join(data_src, directory)):
                img = cv2.imread(os.path.join(data_src, directory, pic))
                img = cv2.resize(img, (32, 32)) 
                #print(img)
                X.append(np.squeeze(np.asarray(img)))
                y.append(directory)
        except:
            pass

    labels = list(set(y))
    label_dict = dict(zip(labels, range(len(labels))))
    Y = np.asarray([label_dict[label] for label in y])
    shuffle_indices = np.random.permutation(np.arange(len(y)))
    x_shuffled = []
    y_shuffled = []
    for index in shuffle_indices:
        x_shuffled.append(X[index])
        y_shuffled.append(Y[index])

    size_of_dataset = len(x_shuffled)
    n_train = int(np.ceil(size_of_dataset * train_test_ratio))
    n_test = int(np.ceil(size_of_dataset * (1 - train_test_ratio)))
    return np.asarray(x_shuffled[0:n_train]), np.asarray(x_shuffled[n_train + 1:size_of_dataset]), np.asarray(y_shuffled[0:n_train]), np.asarray(y_shuffled[
                                                                                                  n_train + 1:size_of_dataset])

## Prepare the Data

In [19]:
train_images, test_images, train_label, test_label = get_train_test_dataset(0.7)

In [20]:
test_images.shape

(14795, 32, 32, 3)

In [21]:
test_images[0]

array([[[143, 146, 131],
        [146, 147, 136],
        [143, 144, 134],
        ...,
        [146, 145, 135],
        [148, 147, 137],
        [144, 143, 133]],

       [[139, 141, 129],
        [149, 149, 137],
        [147, 146, 136],
        ...,
        [144, 143, 133],
        [147, 146, 136],
        [147, 146, 136]],

       [[147, 146, 136],
        [138, 136, 126],
        [156, 151, 144],
        ...,
        [143, 142, 132],
        [146, 145, 135],
        [145, 146, 136]],

       ...,

       [[139, 158, 165],
        [129, 144, 142],
        [144, 146, 136],
        ...,
        [ 79,  74,  73],
        [125, 125, 111],
        [132, 132, 120]],

       [[129, 147, 156],
        [ 87, 105, 107],
        [132, 137, 128],
        ...,
        [108, 108,  92],
        [113, 112, 101],
        [129, 129, 117]],

       [[ 97, 118, 124],
        [112, 134, 135],
        [143, 152, 146],
        ...,
        [107, 106,  95],
        [115, 113, 103],
        [126, 123, 114]]

In [22]:
def _normalize_img(img, label):
    img = tf.cast(img, tf.float32) / 255.
    return (img, label)


In [23]:
def parse_img(img, label):
    return (img, label)

In [24]:
test_dataset = _normalize_img(test_images,test_label)

In [25]:
train_dataset = _normalize_img(train_images,train_label)

In [26]:
test_dataset[0][0] 


<tf.Tensor: shape=(32, 32, 3), dtype=float32, numpy=
array([[[0.56078434, 0.57254905, 0.5137255 ],
        [0.57254905, 0.5764706 , 0.53333336],
        [0.56078434, 0.5647059 , 0.5254902 ],
        ...,
        [0.57254905, 0.5686275 , 0.5294118 ],
        [0.5803922 , 0.5764706 , 0.5372549 ],
        [0.5647059 , 0.56078434, 0.52156866]],

       [[0.54509807, 0.5529412 , 0.5058824 ],
        [0.58431375, 0.58431375, 0.5372549 ],
        [0.5764706 , 0.57254905, 0.53333336],
        ...,
        [0.5647059 , 0.56078434, 0.52156866],
        [0.5764706 , 0.57254905, 0.53333336],
        [0.5764706 , 0.57254905, 0.53333336]],

       [[0.5764706 , 0.57254905, 0.53333336],
        [0.5411765 , 0.53333336, 0.49411765],
        [0.6117647 , 0.5921569 , 0.5647059 ],
        ...,
        [0.56078434, 0.5568628 , 0.5176471 ],
        [0.57254905, 0.5686275 , 0.5294118 ],
        [0.5686275 , 0.57254905, 0.53333336]],

       ...,

       [[0.54509807, 0.61960787, 0.64705884],
        [0.5058

In [27]:
IMG_SIZE = 32 # Specify height and width of image to match the input format of the
BATCH_SIZE = 256 # Big enough to measure an F1-score
AUTOTUNE = tf.data.experimental.AUTOTUNE # Adapt preprocessing and prefetching dynamically to reduce GPU and CPU idle time
SHUFFLE_BUFFER_SIZE = 1024 # Shuffle the training data by a chunck of 1024 observationsmodel
CHANNELS = 3 # Keep RGB color channels to match the input format of the model

In [28]:
def parse_function(filename, label):
    """Function that returns a tuple of normalized image array and labels array.
    Args:
        filename: string representing path to image
        label: 0/1 one-dimensional array of size N_LABELS
    """
    # Read an image from a file
    image_string = tf.io.read_file(filename)
    # Decode it into a dense vector
    image_decoded = tf.image.decode_jpeg(image_string, channels=CHANNELS)
    # Resize it to fixed shape
    image_resized = tf.image.resize(image_decoded, [IMG_SIZE, IMG_SIZE])
    # Normalize it from [0, 255] to [0.0, 1.0]
    image_normalized = image_resized / 255.0
    return image_normalized, label

In [29]:
def create_dataset(filenames, labels, is_training=True):
    """Load and parse dataset.
    Args:
        filenames: list of image paths
        labels: numpy array of shape (BATCH_SIZE, N_LABELS)
        is_training: boolean to indicate training mode
    """
    
    # Create a first dataset of file paths and labels
    dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
    #for element in dataset: 
     #     print(element) 
    # Parse and preprocess observations in parallel
    dataset = dataset.map(parse_img, num_parallel_calls=AUTOTUNE)
    
    if is_training == True:
        # This is a small dataset, only load it once, and keep it in memory.
        dataset = dataset.cache()
        # Shuffle the data each buffer size
        dataset = dataset.shuffle(buffer_size=SHUFFLE_BUFFER_SIZE)
        
    # Batch the data for multiple steps
    dataset = dataset.batch(BATCH_SIZE)
    # Fetch batches in the background while the model is training.
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    
    return dataset

In [30]:
X_test=test_dataset[0]

In [31]:
y_test_bin=test_dataset[1]

In [32]:
test_ds = create_dataset(X_test, y_test_bin)

In [33]:
test_ds

<PrefetchDataset shapes: ((None, 32, 32, 3), (None,)), types: (tf.float32, tf.int32)>

In [34]:
X_train=train_dataset[0]
y_train_bin=train_dataset[1]
train_ds = create_dataset(X_train, y_train_bin)

## Build the Model

In [35]:

IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

c:\users\kuruparans\anaconda3\envs\tf_gpu\lib\site-packages\keras_applications\mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


In [41]:
for image_batch, label_batch in train_ds.take(1):
   pass

image_batch.shape

TensorShape([256, 32, 32, 3])

In [42]:
feature_batch = base_model(image_batch)
print(feature_batch.shape)

(256, 1, 1, 1280)


In [43]:
base_model.trainable = False
base_model.summary()


Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 33, 33, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 16, 16, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 16, 16, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

In [44]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

(256, 1280)


![fig2](https://user-images.githubusercontent.com/18154355/61485417-1cbb1f00-a96f-11e9-8d6a-94964ce8c4db.png)

In [46]:
model = tf.keras.Sequential([
    base_model,
    global_average_layer,
   # tf.keras.layers.Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'),
   # tf.keras.layers.MaxPooling2D(pool_size=2),
   # tf.keras.layers.Dropout(0.3),
   # tf.keras.layers.Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'),
   # tf.keras.layers.MaxPooling2D(pool_size=2),
   # tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation=None), # No activation on final dense layer
    tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)) # L2 normalize embeddings

])

## Train and Evaluate

In [47]:
# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tfa.losses.TripletSemiHardLoss())


In [48]:
# Train the network
history = model.fit(
    train_ds,
    epochs=2000)

Train for 135 steps
Epoch 1/2000
135/135 [==============================] - 15s 110ms/step - loss: 0.9918
Epoch 2/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9885
Epoch 3/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9881
Epoch 4/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9870
Epoch 5/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9866
Epoch 6/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9857
Epoch 7/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9861
Epoch 8/2000
135/135 [==============================] - 2s 18ms/step - loss: 0.9855
Epoch 9/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9861
Epoch 10/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9852
Epoch 11/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9849
Epoch 12/2000
135/135 [=============================

135/135 [==============================] - 3s 19ms/step - loss: 0.9820
Epoch 97/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9816
Epoch 98/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9822
Epoch 99/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9837
Epoch 100/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9829
Epoch 101/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9829
Epoch 102/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9813
Epoch 103/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9820
Epoch 104/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9823
Epoch 105/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9821
Epoch 106/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9826
Epoch 107/2000
135/135 [==============================] - 3s 19ms/step -

135/135 [==============================] - 3s 19ms/step - loss: 0.9818
Epoch 191/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9817
Epoch 192/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9818
Epoch 193/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9808
Epoch 194/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9820
Epoch 195/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9818
Epoch 196/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9818
Epoch 197/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9809
Epoch 198/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9819
Epoch 199/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9820
Epoch 200/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9822
Epoch 201/2000
135/135 [==============================] - 3s 19ms/ste

135/135 [==============================] - 3s 19ms/step - loss: 0.9819
Epoch 286/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9811
Epoch 287/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9816
Epoch 288/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9814
Epoch 289/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9821: 0s - loss: 0.982
Epoch 290/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9815
Epoch 291/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9821
Epoch 292/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9821
Epoch 293/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9819
Epoch 294/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9816
Epoch 295/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9812
Epoch 296/2000
135/135 [===========================

135/135 [==============================] - 3s 19ms/step - loss: 0.9813
Epoch 381/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9816
Epoch 382/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9813
Epoch 383/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9809
Epoch 384/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9812
Epoch 385/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9810
Epoch 386/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9820
Epoch 387/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9810
Epoch 388/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9816
Epoch 389/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9819
Epoch 390/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9813
Epoch 391/2000
135/135 [==============================] - 3s 19ms/ste

...

135/135 [==============================] - 3s 19ms/step - loss: 0.9803
Epoch 1602/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9809
Epoch 1603/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9805
Epoch 1604/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9816
Epoch 1605/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9813
Epoch 1606/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9805
Epoch 1607/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9810
Epoch 1608/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9819
Epoch 1609/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9821: 0s - los
Epoch 1610/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9802
Epoch 1611/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9803
Epoch 1612/2000
135/135 [========================

135/135 [==============================] - 3s 19ms/step - loss: 0.9815
Epoch 1695/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9826
Epoch 1696/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9808
Epoch 1697/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9805: 0s - loss: 0.9
Epoch 1698/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9807
Epoch 1699/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9814
Epoch 1700/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9811
Epoch 1701/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9814: 0s - lo
Epoch 1702/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9804
Epoch 1703/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9804
Epoch 1704/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9801
Epoch 1705/2000
135/135 [=========

135/135 [==============================] - 3s 19ms/step - loss: 0.9813
Epoch 1787/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9807
Epoch 1788/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9812
Epoch 1789/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9810
Epoch 1790/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9813
Epoch 1791/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9811
Epoch 1792/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9808
Epoch 1793/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9807
Epoch 1794/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9811
Epoch 1795/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9810
Epoch 1796/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9809
Epoch 1797/2000
135/135 [==============================] - 

135/135 [==============================] - 3s 19ms/step - loss: 0.9815
Epoch 1881/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9812
Epoch 1882/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9802
Epoch 1883/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9806
Epoch 1884/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9816
Epoch 1885/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9796
Epoch 1886/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9809
Epoch 1887/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9807
Epoch 1888/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9813
Epoch 1889/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9804
Epoch 1890/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9816
Epoch 1891/2000
135/135 [==============================] - 

135/135 [==============================] - 3s 19ms/step - loss: 0.9808
Epoch 1974/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9807
Epoch 1975/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9809
Epoch 1976/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9808
Epoch 1977/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9803
Epoch 1978/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9810
Epoch 1979/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9815
Epoch 1980/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9803
Epoch 1981/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9804
Epoch 1982/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9802
Epoch 1983/2000
135/135 [==============================] - 3s 19ms/step - loss: 0.9811
Epoch 1984/2000
135/135 [==============================] - 

In [49]:
# Evaluate the network
results = model.predict(test_ds)

In [51]:
# Save test embeddings for visualization in projector
np.savetxt("vecsMobi.tsv", results, delimiter='\t')

out_m = io.open('metaMobi.tsv', 'w', encoding='utf-8')
for img, labels in tfds.as_numpy(test_ds):
    [out_m.write(str(x) + "\n") for x in labels]
out_m.close()


try:
  from google.colab import files
  files.download('vecsMobi.tsv')
  files.download('metaMobi.tsv')
except:
  pass

## Embedding Projector

The vector and metadata files can be loaded and visualized here: https://projector.tensorflow.org/

You can see the results of our embedded test data when visualized with UMAP:
![embedding](https://user-images.githubusercontent.com/18154355/61600295-e6470380-abfd-11e9-8a00-2b25e7e6916f.png)
